In [2]:
from collections import namedtuple
from bisect import bisect_right

from string import ascii_lowercase

In [3]:
# Utilities associated with list of key-value pairs ordered by the key.
KeyAndValue = namedtuple("KeyAndValue", field_names=["key", "value"])

def insert_into_kv_list(kv_list, key, value):
    i = bisect_right(kv_list, key, key=lambda kv: kv.key)  # kv_list[i - 1].key <= key < kv_list[i].key
    if (i > 0) and (kv_list[i - 1].key == key):
        # key was already present
        kv_list[i - 1] = KeyAndValue(key=key, value=value)
    else:
        kv_list.insert(i, KeyAndValue(key=key, value=value))

def search_in_kv_list(kv_list, key):
    i = bisect_right(kv_list, key, key=lambda kv: kv.key)  # kv_list[i - 1].key <= key < kv_list[i].key
    if (i > 0) and (kv_list[i - 1].key == key):
        return kv_list[i - 1]
    else:
        return None

def delete_from_kv_list(kv_list, key):
    """Returns True if key was found and deleted, returns False otherwise."""
    i = bisect_right(kv_list, key, key=lambda kv: kv.key)  # kv_list[i - 1].key <= key < kv_list[i].key
    if (i > 0) and (kv_list[i - 1].key == key):
        kv_list.pop(i - 1)
        return True
    else:
        return False


In [4]:
class StorageOfNames():
    """Storage of names with their corresponding data implemented as B+ tree.
    Names are hashed and B+ tree is indexed by those hashes. 
    Because several names could correspond to a single hash value, 
    a list of (name, data) pairs is associated with each hash value in the leaf node.
    
    Parameters
    ----------
    d : int
        Half-order of the b+ tree, each node (except the root) should contain from d to 2d entries.
    calc_hash : funciton
        Hash funciton used to hash names.
    calc_upper_hash : function
        Is only required by search_less_than and search_greater_than functionalities.
        upper_hash(name) is an upper bound for hash values of names which are less or equal than the name.
    calc_lower_hash : funciton
        Is only required by search_less_than and search_greater_than functionalities.
        lower_hash(name) is a lower bound for hash values of names which are greater or equal than the name.

    Attributes
    ----------
    root : Node
        The root node of the B+ tree.
    
    """

    def __init__(self, d, calc_hash, calc_upper_hash=None, calc_lower_hash=None):
        self.d = d
        self.root = Node(is_leaf=True, d=d, keys=[], keys_data=[])
        self.calc_hash = calc_hash
        self.calc_upper_hash = calc_upper_hash
        self.calc_lower_hash = calc_lower_hash

    def __repr__(self):
        """Displays structure of the underlying B+ tree."""
        return self._display_node(self.root)
    
    def _display_node(self, node, tab_count=0):
        """Display the structure of the node as follows:
        [name_hashes]
            [name_hashes]
                name_hash : [names]

                name_hash : [names]

            [name_hashes]
                name_hash : [names]

                name_hash : [names]

        """
        if node.is_leaf:
            repr = ""
            for name_hash, kv_list in zip(node.keys, node.keys_data):
                names = [name for name, _ in kv_list]
                repr = repr + "\t" * tab_count + f"{name_hash} : {names}\n"
            return repr + "\n"
        else:
            repr = "\t" * tab_count + f"{node.keys}\n"
            for child in node.children:
                repr = repr + self._display_node(child, tab_count=tab_count + 1)
            return repr
    
    def insert(self, name, data):
        """Inserts the name with the corresponding data."""
        # Search for the leaf node to insert the name into
        name_hash  = self.calc_hash(name)        
        leaf_node = self.root.search(name_hash)

        # Find the place for name_hash
        i = bisect_right(leaf_node.keys, name_hash)  # keys[i - 1] <= name_hash < keys[i]
        if (i > 0) and (name_hash == leaf_node.keys[i - 1]):
            # hash was already present
            insert_into_kv_list(leaf_node.keys_data[i - 1], key=name, value=data)
        else:
            # hash was not present
            leaf_node.keys.insert(i, name_hash)
            leaf_node.keys_data.insert(i, [KeyAndValue(key=name, value=data)])

        # Handle overflow
        new_root = leaf_node.handle_overflow()
        if new_root is not None:
            self.root = new_root

    def search(self, name):
        """Returns data associated with the given name."""
        # search for the leaf node that might contain the name
        name_hash = self.calc_hash(name)
        leaf_node = self.root.search(name_hash)

        # check if name_hash is present in the leaf node
        i = bisect_right(leaf_node.keys, name_hash)  # keys[i - 1] <= name_hash < keys[i]
        if (i > 0) and (leaf_node.keys[i - 1] == name_hash):
            kv_list = leaf_node.keys_data[i - 1]
        else:
            return f"StorageOfNames.search: \"{name}\" was not found."

        # search for the name in the key-value list associated with the name's hash
        kv = search_in_kv_list(kv_list, key=name)
        if kv is not None:
            return kv.value
        else:
            return f"StorageOfNames.search: \"{name}\" was not found."
    
    def search_less_than(self, name, strict=True):
        """Search for all names that are lexicographically less than the given name.
        
        Parameters
        ----------
        name : string
        strict : bool
            If True, then search for names strictly less than the given name.
            If False, then search for names less or equal than the given name.

        Returns
        -------
        names_and_data : list of key-value pairs
            Searched names with their corresponding data.
        
        """
        def is_less(another_name):
            return (another_name < name) or ((not strict) and (another_name == name))

        # all names less or equal than name can't have hash greater than upper_hash
        upper_hash = self.calc_upper_hash(name)
        # all names that have hash less than lower_hash are less than the given name
        lower_hash = self.calc_lower_hash(name)

        # Scan the leaf level from right to left
        # Collect all the names with hash between lower_hash and upper_hash that are less than the given name
        names_and_data = []
        node = self.root.search(upper_hash)
        key_index = len(node.keys) - 1
        while (node is not None) and (node.keys[key_index] >= lower_hash):
            # filter the needed names
            new_names_and_data = filter(lambda name_and_data: is_less(name_and_data.key), 
                                        node.keys_data[key_index])
            names_and_data.extend(new_names_and_data)

            # move left
            if key_index > 0:
                key_index -= 1
            else:
                node = node.left
                if node is not None:
                    key_index = len(node.keys) - 1

        # Collect all the names with hash less than lower_hash
        while (node is not None):
            names_and_data.extend(node.keys_data[key_index])

            # move left
            if key_index > 0:
                key_index -= 1
            else:
                node = node.left
                if node is not None:
                    key_index = len(node.keys) - 1
        
        return names_and_data

    def search_greater_than(self, name, strict=True):
        """Search for all names that are lexicographically greater than the given name.
        
        Parameters
        ----------
        name : string
        strict : bool
            If True, then search for names strictly greater than the given name.
            If False, then search for names greater or equal than the given name.

        Returns
        -------
        names_and_data : list of key-value pairs
            Searched names with their corresponding data.
        
        """
        def is_greater(another_name):
            return (another_name > name) or ((not strict) and (another_name == name))

        # all names greater or equal than name can't have hash less than lower_hash
        lower_hash = self.calc_lower_hash(name)
        # all names that have hash greater than upper_hash are greater than the given name
        upper_hash = self.calc_upper_hash(name)

        # Scan the leaf level from left to right
        # Collect all the names with hash between lower_hash and upper_hash that are greater than the given name
        names_and_data = []
        node = self.root.search(lower_hash)
        key_index = 0
        while (node is not None) and (node.keys[key_index] <= upper_hash):
            # filter the needed names
            new_names_and_data = filter(lambda name_and_data: is_greater(name_and_data.key), 
                                        node.keys_data[key_index])
            names_and_data.extend(new_names_and_data)

            # move right
            if key_index < len(node.keys_data) - 1:
                key_index += 1
            else:
                node = node.right
                key_index = 0

        # Collect all the names with hash less than lower_hash
        while (node is not None):
            names_and_data.extend(node.keys_data[key_index])

            # move right
            if key_index < len(node.keys_data) - 1:
                key_index += 1
            else:
                node = node.right
                key_index = 0
        
        return names_and_data
    
    def delete(self, name):
        """Deletes an entry associated with the given name."""
        # Search for the leaf node that might contain the name
        name_hash = self.calc_hash(name)
        leaf_node = self.root.search(name_hash)

        # Remove the name from the leaf node
        i = bisect_right(leaf_node.keys, name_hash)  # keys[i - 1] <= name_hash < keys[i]
        if (i > 0) and (leaf_node.keys[i - 1] == name_hash):
            was_deleted = delete_from_kv_list(leaf_node.keys_data[i - 1], key=name)
            if not was_deleted:
                return f"StorageOfNames.delete: \"{name}\" was not found."
        else:
            return f"StorageOfNames.delete: \"{name}\" was not found."

        if len(leaf_node.keys_data[i - 1]) > 0:
            # name_hash still has associated with it names
            return

        # name_hash has no associated names in the tree, remove name_hash entirely from the tree
        # Search for occurence of name_hash in internal node
        internal_node, key_pos = self.root._search_in_internal(key=name_hash)

        if internal_node is None:
            # name_hash is present only in the leaf node
            leaf_node.keys.pop(i - 1)
            leaf_node.keys_data.pop(i - 1)

            new_root = leaf_node.handle_underflow()
        else:
            # name_hash is present in internal node and leaf node.
            # Hence name_hash will be the first key in the leaf node.

            # Consider 2 cases: height(internal_node) = 1; height(internal_node) > 1
            if leaf_node.parent == internal_node:
                if len(leaf_node.keys) == 1:
                    internal_node.children.pop(key_pos + 1)
                    internal_node.keys.pop(key_pos)

                    # handle the leaf level
                    left, right = leaf_node.left, leaf_node.right  # left won't be None
                    if right is not None:
                        left.right = right
                        right.left = left
                    else:
                        left.right = None
                    
                    new_root = internal_node.handle_underflow()
                else:
                    leaf_node.keys.pop(0)
                    leaf_node.keys_data.pop(0)
                    internal_node.keys[key_pos] = leaf_node.keys[0]

                    new_root = leaf_node.handle_underflow()
            else:
                # leaf_node will be the first child of the parent node
                parent = leaf_node.parent

                if len(leaf_node.keys) == 1:
                    internal_node.keys[key_pos] = leaf_node.right.keys[0]
                    parent.children.pop(0)
                    parent.keys.pop(0)

                    # handle the leaf level
                    left, right = leaf_node.left, leaf_node.right  # right won't be None
                    if left is not None:
                        left.right = right
                        right.left = left
                    else:
                        right.left = None

                    new_root = parent.handle_underflow()
                else:
                    leaf_node.keys.pop(0)
                    leaf_node.keys_data.pop(0)
                    internal_node.keys[key_pos] = leaf_node.keys[0]

                    new_root = leaf_node.handle_underflow()
        
        if new_root is not None:
            self.root = new_root

class Node():
    """The node of B+ tree.

    Parameters
    ----------
    is_leaf : bool
        True if the node is leaf, False if the node is internal.
    d : int
        Half-order of the node, each node (except the root, i.e. the one without the parent) should contain from d to 2d entries.
    keys : array-like
        Ordered list of keys in the current node.
    children : array-like
        If internal node, then the list of number_of_keys + 1 children nodes;
        If leaf node, then None.
    left : Node
        If internal node, then None.
        If leaf node, then the leaf node on the left.
    right : Node
        If internal node, then None.
        If leaf node, then the leaf node on the right.
    parent : Node
        Parent node.
    keys_data : array-like
        If leaf node, then contains the data associated with each key.
        If internal node, then None.

    """
    
    def __init__(self, is_leaf, d, keys, children=None, left=None, right=None, parent=None, keys_data=None):
        self.is_leaf = is_leaf
        self.d = d
        self.keys = keys
        self.children = children
        self.left = left
        self.right = right
        self.parent = parent
        self.keys_data = keys_data

    def search(self, key):
        """Returns the leaf node that might contain the given key."""
        if self.is_leaf:
            return self
        
        i = bisect_right(self.keys, key)  # keys[i - 1] <= key < keys[i]
        return self.children[i].search(key)

    def handle_overflow(self):
        """If number of keys in the node is greater than 2d, then split happens and the middle key is propagated upwards.
        
        Returns
        -------
        If the root hasn't changed, then returns None.
        If the root has changed, then returns the new root node.

        """
        # check if overflow happened
        if len(self.keys) <= 2 * self.d:
            return None
        
        parent = self.parent
        # split the node (consider leaf and non-leaf cases)
        # left is a modified version of the current node, right is a new node
        propagated_key, left, right = self._split()

        # insert propagated value into parent (consider root and non-root cases)
        if parent is not None:
            i = bisect_right(parent.keys, propagated_key)  # keys[i - 1] <= propagated_key < keys[i]
            parent.keys.insert(i, propagated_key)
            parent.children.insert(i + 1, right)

            # overflow in parent might occur, so handle it
            return parent.handle_overflow()
        else:
            new_root = Node(is_leaf=False, d=self.d, 
                            keys=[propagated_key],
                            children=[left, right])
            left.parent = new_root
            right.parent = new_root
            return new_root

    def _split(self):
        """Splits the overflowed node (with 2d+1 keys) into 2 nodes. 
        Left node contains first d keys, right node contains the last d+1 keys.
        If internal node, then the first key of the right node is extracted, otherwise stays in place. Current node is being modified to become the left node.

        Returns
        -------
        propagated_key : type that supports <, >, =
            The middle key to be propagated upwards.
        left : Node
        right : Node

        """
        if self.is_leaf:
            propagated_key = self.keys[self.d]
            
            right = Node(is_leaf=True, d=self.d, 
                         keys=self.keys[self.d:],
                         left=self, right=self.right,
                         parent=self.parent,
                         keys_data=self.keys_data[self.d:])
            # handle the leaf level
            if self.right is not None:
                self.right.left = right

            # make current node the left node
            self.keys = self.keys[:self.d]
            self.right = right
            self.keys_data = self.keys_data[:self.d]
            left = self
        else:
            propagated_key = self.keys[self.d]

            right = Node(is_leaf=False, d=self.d,
                         keys=self.keys[self.d + 1:],
                         children=self.children[self.d + 1:],
                         parent=self.parent)
            # update the parent of the children
            for child in right.children:
                child.parent = right

            # make current node the left node
            self.keys = self.keys[:self.d]
            self.children = self.children[:self.d + 1]
            left = self

        return propagated_key, left, right

    def _search_in_internal(self, key):
        """Search for internal node that contains the given key.

        Returns
        -------
        (internal_node, key_position) : tuple
            If the key is found in internal node, then returns the node and position of the key in the node.
            Otherwise returns (None, None).
        
        """
        if self.is_leaf:
            return None, None
        
        i = bisect_right(self.keys, key)  # keys[i - 1] <= key < keys[i]
        if (i > 0) and (self.keys[i - 1] == key):
            return self, i - 1
        else:
            return self.children[i]._search_in_internal(key)
    
    def handle_underflow(self):
        """If number of keys is less than d, then redistribute or merge and propagate upwards.
        
        Returns
        -------
        new_root : Node
            If tree has decreased its height, the new root node is returned.
            Otherwise None is returned.
        
        """
        # Consider the root case first
        if self.parent is None:
            if len(self.keys) > 0:
                return None
            else:
                if self.is_leaf:
                    # Empty tree
                    return self
                else:
                    self.children[0].parent = None
                    return self.children[0]
            
        if len(self.keys) < self.d:
            was_redistributed = self._try_to_redistribute()

            if was_redistributed:
                return None
            else:
                parent = self.parent
                self._merge()
                return parent.handle_underflow()

    def _try_to_redistribute(self):
        """Current non-root node has d-1 keys. If left or right sibling has at least d+1 keys, 
        move 1 key to this node. 
        
        Returns 
        -------
        was_redistributed : bool
            True if redistribution was possible, False otherwise.
        
        """
        parent = self.parent
        if len(self.keys) > 0:
            i = bisect_right(parent.keys, self.keys[0])  # keys[i - 1] <= key < keys[i]
        else:
            i = parent.children.index(self)
        left_sibling  = parent.children[i - 1] if (i > 0) else None
        right_sibling = parent.children[i + 1] if (i < len(parent.keys)) else None

        if self.is_leaf:
            if (left_sibling is not None) and (len(left_sibling.keys) > self.d):
                # Since left sibling is present, remember that parent.keys[i - 1] = self.keys[0]
                self.keys.insert(0, left_sibling.keys.pop())
                self.keys_data.insert(0, left_sibling.keys_data.pop())

                parent.keys[i - 1] = self.keys[0]
                return True
            elif (right_sibling is not None) and (len(right_sibling.keys) > self.d):
                # remember that parent.keys[i] = right_sibling.keys[0]
                self.keys.append(right_sibling.keys.pop(0))
                self.keys_data.append(right_sibling.keys_data.pop(0))

                parent.keys[i] = right_sibling.keys[0]
                return True
            else:
                return False
        else:
            # remember that any two internal nodes don't contain the same key
            if (left_sibling is not None) and (len(left_sibling.keys) > self.d):
                self.keys.insert(0, parent.keys[i - 1])
                self.children.insert(0, left_sibling.children.pop())

                parent.keys[i - 1] = left_sibling.keys.pop()
                return True
            elif (right_sibling is not None) and (len(right_sibling.keys) > self.d):
                self.keys.append(parent.keys[i])
                self.children.append(right_sibling.children.pop(0))

                parent.keys[i] = right_sibling.keys.pop(0)
                return True
            else:
                return False

    
    def _merge(self):
        """Current non-root node has d-1 keys. Left and right siblings (if present) have exactly d keys.
        Merge the node with its left or right sibling."""
        parent = self.parent
        if len(self.keys) > 0:
            i = bisect_right(parent.keys, self.keys[0])  # keys[i - 1] <= key < keys[i]
        else:
            i = parent.children.index(self)
        left_sibling  = parent.children[i - 1] if (i > 0) else None
        right_sibling = parent.children[i + 1] if (i < len(parent.keys)) else None

        if self.is_leaf:
            if left_sibling is not None:
                left_sibling.keys.extend(self.keys)
                left_sibling.keys_data.extend(self.keys_data)

                # handle leaf lefel
                if self.right is not None:
                    self.right.left = left_sibling
                    left_sibling.right = self.right
                else:
                    left_sibling.right = None

                parent.keys.pop(i - 1)
                parent.children.pop(i) 
            else:
                self.keys.extend(right_sibling.keys)
                self.keys_data.extend(right_sibling.keys_data)

                # handle leaf level
                if right_sibling.right is not None:
                    right_sibling.right.left = self
                    self.right = right_sibling.right
                else:
                    self.right = None
                
                parent.keys.pop(i)
                parent.children.pop(i + 1)
        else:
            if left_sibling is not None:
                left_sibling.keys.append(parent.keys.pop(i - 1))
                left_sibling.keys.extend(self.keys)
                left_sibling.children.extend(self.children)

                # update the parent of self's children
                for child in self.children:
                    child.parent = left_sibling

                parent.children.pop(i)
            else:
                self.keys.append(parent.keys.pop(i))
                self.keys.extend(right_sibling.keys)
                self.children.extend(right_sibling.children)

                # update the parent of right_sibling's children
                for child in right_sibling.children:
                    child.parent = self
                
                parent.children.pop(i + 1)

In [5]:
def char_hash(c):
    return ord(c.lower()) - 97

def nfirst_hash(name, n=3, N=len(ascii_lowercase), char_hash=char_hash):
    """Hash value based on the first n characters of the name.
    
    Parameters
    ----------
    name : string
    n : int
        Number of first characters in the name based on which the hash is calculated.
    N : int
        Number of allowed characters from which the name could consist.
        By default it is assumed that the name consists only from english letters.
    char_hash : funciton
        Function used to hash characters. It must have the range {0, 1, ..., N - 1}.

    """
    result = 0
    power = 1
    for i in range(n - 1, -1, -1):
        if i < len(name):
            result += char_hash(name[i]) * power
        power *= N
    
    return result

#### Simple tests

In [6]:
n = 1   # number of first characters for the first part of the hash
p = 0  # number of bits in the second part of the hash
storage = StorageOfNames(
    d=1,  # b+ tree of order 2
    calc_hash      =lambda name: nfirst_hash(name, n),
    calc_lower_hash=lambda name: nfirst_hash(name, n),
    calc_upper_hash=lambda name: nfirst_hash(name, n)
)

names = ["a", "b", "e", "c", "d", "f"]
for name in names:
    storage.insert(name=name, data=f"_{name}_")
storage.insert(name="cc", data="_cc_")
print(storage)

[2]
	[1]
		0 : ['a']

		1 : ['b']

	[3, 4]
		2 : ['c', 'cc']

		3 : ['d']

		4 : ['e']
		5 : ['f']




In [7]:
print(storage.search(name="cc"))
print(storage.search(name="dd"))

_cc_
StorageOfNames.search: "dd" was not found.


In [8]:
print(storage.search_less_than(name="cc"), '\n')
print(storage.search_greater_than(name="cc"))

[KeyAndValue(key='c', value='_c_'), KeyAndValue(key='b', value='_b_'), KeyAndValue(key='a', value='_a_')] 

[KeyAndValue(key='d', value='_d_'), KeyAndValue(key='e', value='_e_'), KeyAndValue(key='f', value='_f_')]


In [9]:
print((storage))
storage.delete("b")
print(storage)

[2]
	[1]
		0 : ['a']

		1 : ['b']

	[3, 4]
		2 : ['c', 'cc']

		3 : ['d']

		4 : ['e']
		5 : ['f']


[3]
	[2]
		0 : ['a']

		2 : ['c', 'cc']

	[4]
		3 : ['d']

		4 : ['e']
		5 : ['f']




In [10]:
storage.delete("a")
print(storage)

[3, 4]
	2 : ['c', 'cc']

	3 : ['d']

	4 : ['e']
	5 : ['f']




In [11]:
storage.delete("c")
print(storage)

[3, 4]
	2 : ['cc']

	3 : ['d']

	4 : ['e']
	5 : ['f']




### Tests with order 4 tree filled with real names

In [12]:
names = [
    "Sophia", "Liam", "Olivia", "Noah", "Emma", "Oliver", "Ava", "Elijah", "Isabella", "William",
    "Mia", "James", "Charlotte", "Benjamin", "Amelia", "Lucas", "Harper", "Henry", "Evelyn", "Alexander",
    "Abigail", "Mason", "Emily", "Daniel", "Elizabeth", "Jacob", "Sofia", "Michael", "Madison", "Ethan",
    "Avery", "Matthew", "Ella", "Joseph", "Scarlett", "Samuel", "Grace", "David", "Chloe", "John",
    "Victoria", "Jackson", "Penelope", "Carter", "Riley", "Sebastian", "Layla", "Jayden", "Lily", "Gabriel",
    "Zoey", "Owen", "Nora", "Dylan", "Hannah", "Luke", "Eleanor", "Anthony", "Leah", "Isaac",
    "Addison", "Christopher", "Sarah", "Joshua", "Stella", "Andrew", "Paisley", "Ryan", "Violet", "Nathan",
    "Mila", "Caleb", "Hazel", "Logan", "Aurora", "Adrian", "Luna", "Hunter", "Samantha", "Thomas",
    "Maya", "Julian", "Caroline", "Aaron", "Kennedy", "Jeremiah", "Allison", "Adam", "Willow", "Charles",
    "Sadie", "Connor", "Julia", "Eli", "Brynn", "Robert", "Alice", "Angel", "Claire", "Gavin", "Audrey",
    "Evan", "Eliana", "Brandon", "Lucy", "Tyler", "Anna", "Jonathan", "Bella", "Nicholas", "Jade"
]
len(names)

111

In [13]:
def _hash(string, p=16):
    """Returns the hash value of the string in range {0, 1, ..., 2^p - 1}."""
    mod = 1 << p
    result = 0
    prime = 31
    for c in string:
        result = (result * prime + ord(c)) % mod
    
    return result

    # Or just use python's hash algorithm:
    # return hash(string) % (1 << p)

In [14]:
# Hash function will keep the ordering by first n letters of the name
n = 3   # number of first letters for the first part of the hash
p = 16  # number of bits for the second part of the hash
storage = StorageOfNames(
    d=2,
    calc_hash      =lambda name: nfirst_hash(name, n) * (1 << p) + _hash(name),
    calc_lower_hash=lambda name: nfirst_hash(name, n) * (1 << p),
    calc_upper_hash=lambda name: (nfirst_hash(name, n) + 1) * (1 << p),
)


In [15]:
for name in names:
    storage.insert(name, f"_{name}_")

In [16]:
storage

[399542564]
	[23444313, 89383115, 133761471, 213304909]
		[5329034, 19035462, 20745803]
			1165713 : ['Aaron']
			2287557 : ['Abigail']
			5115823 : ['Adam']

			5329034 : ['Addison']
			6264679 : ['Adrian']

			19035462 : ['Alexander']
			19310176 : ['Alice']
			19517834 : ['Allison']

			20745803 : ['Amelia']
			22366093 : ['Andrew']
			22579169 : ['Angel']
			23017152 : ['Anna']

		[35209088, 52002864]
			23444313 : ['Anthony']
			34335766 : ['Audrey']

			35209088 : ['Aurora']
			35783340 : ['Ava']
			36054263 : ['Avery']
			51878814 : ['Bella']

			52002864 : ['Benjamin']
			73280390 : ['Brandon']
			74888585 : ['Brynn']

		[100553676, 101257373]
			89383115 : ['Caleb']
			89730029 : ['Carter']
			89754031 : ['Caroline']

			100553676 : ['Charlotte']
			100589284 : ['Charles']

			101257373 : ['Chloe']
			101648087 : ['Christopher']
			107367524 : ['Claire']
			113348143 : ['Connor']

		[174579908, 196481154, 196518479, 198444124]
			133761471 : ['Daniel']
			134291028 : ['David']

In [17]:
print(storage.search("Emma"))
print(storage.search("Artem"))

_Emma_
StorageOfNames.search: "Artem" was not found.


In [18]:
print(storage.search_less_than("Alice"))
print(storage.search_greater_than("W"))

[KeyAndValue(key='Alexander', value='_Alexander_'), KeyAndValue(key='Adrian', value='_Adrian_'), KeyAndValue(key='Addison', value='_Addison_'), KeyAndValue(key='Adam', value='_Adam_'), KeyAndValue(key='Abigail', value='_Abigail_'), KeyAndValue(key='Aaron', value='_Aaron_')]
[KeyAndValue(key='William', value='_William_'), KeyAndValue(key='Willow', value='_Willow_'), KeyAndValue(key='Zoey', value='_Zoey_')]


In [19]:
print(storage.search("Anthony"))
storage.delete("Anthony")
print(storage.search("Anthony"))

_Anthony_
StorageOfNames.search: "Anthony" was not found.
